In [ ]:
from dotenv import load_dotenv  # Import to load environment variables
import pandas as pd  # Import pandas for data manipulation
import matplotlib.pyplot as plt  # Import for plotting results
import numpy as np  # Import NumPy for numerical operations
from sklearn.ensemble import RandomForestClassifier  # Import RandomForest for machine learning
from sklearn.model_selection import train_test_split  # Import to split data into training/testing sets
from eth_rpc import PrivateKeyWallet  # Import for Ethereum wallet management
from emp_orderly.utils import from_address  # Import utility to get address from wallet
from emp_orderly import (
    Strategy, EmpOrderly,  # Import core trading strategy and backtesting classes
    crossover, plot_heatmaps,  # Import additional utilities for analysis and plotting
    EMA, SMA, SLOPE, CHOP,  # Import various technical indicators
    EmpyrealOrderlySDK,  # Import SDK for Empyreal Orderly
)
from emp_orderly_types import PerpetualAssetType, Interval, OrderType  # Import type definitions for trading

# Load environment variables from a .env file
load_dotenv()

# Initialize wallet and SDK for trading
wallet = PrivateKeyWallet.create_new()  # Create a new private key wallet
orderly_id = from_address(wallet.address)  # Get the wallet address
sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)  # Initialize the SDK

# Manual implementation of the Relative Strength Index (RSI)
def RSI(close, period=14):
    # Calculate the differences in closing prices
    delta = np.diff(close)
    # Separate gains and losses
    gain = np.where(delta > 0, delta, 0)  # Positive price changes
    loss = np.where(delta < 0, -delta, 0)  # Negative price changes

    # Calculate the average gain and loss over the specified period
    avg_gain = np.convolve(gain, np.ones((period,)) / period, mode='valid')
    avg_loss = np.convolve(loss, np.ones((period,)) / period, mode='valid')

    # Calculate the RSI using average gain and loss
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Pad the resulting RSI with NaNs to match the length of the input close array
    return np.concatenate([np.full((period,), np.nan), rsi])

# Define the trading strategy class that inherits from the Strategy base class
class MLBasedStrategy(Strategy):
    # Define parameters for the strategy
    n1: int = 10  # Short-term SMA period
    n2: int = 40  # Long-term SMA period
    rsi_period: int = 14  # Period for RSI calculation
    risk_tolerance: float = 0.02  # Stop-loss tolerance set to 2% of the position size

    def init(self):
        # Initialize the strategy indicators
        close = self.data.close  # Get the closing price data
        self.sma1 = self.I(SMA, close, self.n1)  # Calculate the short-term SMA
        self.sma2 = self.I(SMA, close, self.n2)  # Calculate the long-term SMA
        self.rsi = self.I(RSI, close, self.rsi_period)  # Calculate the RSI

        # Prepare features for machine learning model training (SMA1, SMA2, RSI)
        features = pd.DataFrame({
            'SMA1': self.sma1,
            'SMA2': self.sma2,
            'RSI': self.rsi,
            'Close': close
        }).dropna()  # Remove rows with NaN values

        # Define target labels for machine learning: Buy (1), Sell (-1), Hold (0)
        features['Target'] = np.where((features['SMA1'] > features['SMA2']) & (features['RSI'] < 30), 1,
                                      np.where((features['SMA1'] < features['SMA2']) & (features['RSI'] > 70), -1, 0))

        # Split the dataset into training and testing sets
        X = features[['SMA1', 'SMA2', 'RSI']]  # Features for training
        y = features['Target']  # Target variable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)  # 80-20 split

        # Train the Random Forest machine learning model
        self.model = RandomForestClassifier(n_estimators=100)  # Initialize model with 100 trees
        self.model.fit(X_train, y_train)  # Fit the model on training data

    def next(self):
        # Retrieve the latest data for making predictions
        close = self.data.close[-1]  # Latest closing price
        sma1 = self.sma1[-1]  # Latest short-term SMA
        sma2 = self.sma2[-1]  # Latest long-term SMA
        rsi = self.rsi[-1]  # Latest RSI value

        # Use the trained model to predict the trading signal
        prediction = self.model.predict([[sma1, sma2, rsi]])

        # Implement risk management with stop-loss conditions
        if self.position.is_long and close < self.position.entry_price * (1 - self.risk_tolerance):
            self.position.close()  # Close long position if price falls below stop-loss
        elif self.position.is_short and close > self.position.entry_price * (1 + self.risk_tolerance):
            self.position.close()  # Close short position if price rises above stop-loss

        # Execute trading decisions based on ML predictions
        if prediction == 1 and not self.position.is_long:  # If prediction is Buy and not already long
            self.position.close()  # Close any existing positions
            self.buy(size=0.5)  # Open a new long position
        elif prediction == -1 and not self.position.is_short:  # If prediction is Sell and not already short
            self.position.close()  # Close any existing positions
            self.sell(size=0.5)  # Open a new short position

# Initialize the backtester with initial cash and settings
tester = EmpOrderly(
    cash=1000,  # Starting cash amount
    commission=.0001,  # Trading commission
    exclusive_orders=True,  # Prevent overlapping orders
    sdk=sdk,  # Use the initialized SDK
)

# Load the trading strategy and historical data
tester.set_strategy(MLBasedStrategy)  # Set the defined strategy for testing
await tester.load_data(
    lookback=12,  # Number of previous periods to consider
    interval=Interval.fifteen_minute,  # Time interval for data
    asset=PerpetualAssetType.BTC,  # Asset type for trading
)

# Run the backtest to evaluate the strategy's performance
tester.backtest()

# Plot the backtesting results for analysis
tester.plot(show_price_data=False)  # Plot without showing raw price data
plt.show()  # Display the plot
